### Implementing a simple 2-layered GCN from scratch in Pytorch
Associated Paper: https://arxiv.org/pdf/1609.02907.pdf
Example Taken: 2-layered GCN on CORA Dataset

_TODO: Modularize GCN class_

In [13]:
import torch
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
import pandas as pd
import os

In [14]:
data_dir = "data/cora/"
edgelist = pd.read_csv(os.path.join(data_dir, "cora.cites"), sep='\t', header=None, names=["target", "source"])

In [15]:
column_names = ["paper_id"] + [f"term_{idx}" for idx in range(1433)] + ["subject"]
papers = pd.read_csv(
    os.path.join(data_dir, "cora.content"), sep="\t", header=None, names=column_names,
)
print("Papers shape:", papers.shape)

Papers shape: (2708, 1435)


In [16]:
# Paper Index -> Id & Vice Verca
paper_idx_to_id = {idx_: id_ for idx_, id_ in enumerate(list(set(papers["paper_id"])))}

# Paper Id -> Index
paper_id_to_idx = {id_: idx_ for idx_, id_ in paper_idx_to_id.items()}

In [17]:
# Calculate Normalized Adjacency Matrix -> A_HAT

# Create an empty Adjacency matrix
A = torch.zeros(papers.shape[0], papers.shape[0])

# Fill the adjacency matrix wherever there is an edge
for pair in edgelist.values:
    A[paper_id_to_idx[pair[0]], paper_id_to_idx[pair[1]]] = 1

# Create an Identity Matrix for A
I = torch.eye(A.shape[0])

# A_TILDA = A + I
A_TILDA = A + I

# Create the Inverse Squared Diagonal Matrix of A_TILDA
D_TILDA_INVERSE_SQUARED = torch.zeros_like(A_TILDA)

for i in range(len(A_TILDA)):
    D_TILDA_INVERSE_SQUARED[i, i] = A_TILDA[i].sum().pow(-0.5)

# Finally A_HAT = D_TILDA_INVERSE_SQUARED @ A_TILDA @ D_TILDA_INVERSE_SQUARED
A_HAT = D_TILDA_INVERSE_SQUARED @ A_TILDA @ D_TILDA_INVERSE_SQUARED

In [18]:
# Create a two layered GCN model
A_HAT.shape

torch.Size([2708, 2708])

In [19]:
# CREATE X & Y: Preparing Dataset
dataset = papers.values
pairwise_indices = [x[0] for x in dataset]
X_PRE = []
Y_PRE = []
for i in range(len(dataset)):
    idx_ = pairwise_indices.index(paper_idx_to_id[i])    
    X_PRE.append(list(dataset[idx_][1:-1]))
    Y_PRE.append(dataset[idx_][-1])

X = torch.tensor(X_PRE, dtype=torch.float32)

# Convert Y to Onehot

y_idx_to_label = {idx_: label_ for idx_, label_ in enumerate(set(Y_PRE))}
y_label_to_idx = {label_: idx_ for idx_, label_ in y_idx_to_label.items()}

Y_ONE_HOT = torch.zeros(len(Y_PRE), len(set(Y_PRE)))
for idx_ in range(len(Y_PRE)):
    row = torch.zeros(7)
    row[y_label_to_idx[Y_PRE[idx_]]] = 1
    Y_ONE_HOT[idx_] = row

In [20]:
# Define Model Weights
feature_vector_size = X.shape[1]
hidden_layer_size = 100
output_size = len(set(Y_PRE))

# Weights at layer 0, 1
W_0 = torch.randn(feature_vector_size, hidden_layer_size, dtype=torch.float32, requires_grad=True)
W_1 = torch.randn(hidden_layer_size, output_size, dtype=torch.float32, requires_grad=True)

In [22]:
epochs = 1000
lr = 0.01

for epoch in range(epochs):
    # DO FORWARD PASS
    # Calculate first hidden layer outputs
    H_1 = ((A_HAT @ X) @ W_0).relu()  # H_1 reduces feature vector space from 1433 -> 100

    # Calculate Output from our 2-layer GCN
    O = ((A_HAT @ H_1) @ W_1).relu()

    # Calculate loss
    loss = CrossEntropyLoss()
    model_loss = loss(O, Y_ONE_HOT)

    print(f"Epoch: {epoch}; Model Loss: {model_loss.item()}")

    W_0.grad = None
    W_1.grad = None

    # DO BACKWARD PASS
    model_loss.backward()
    W_0.data -= lr * W_0.grad
    W_1.data -= lr * W_1.grad

Epoch: 0; Model Loss: 5.46131706237793
Epoch: 1; Model Loss: 5.405853271484375
Epoch: 2; Model Loss: 5.351841449737549
Epoch: 3; Model Loss: 5.299158573150635
Epoch: 4; Model Loss: 5.247714519500732
Epoch: 5; Model Loss: 5.1974263191223145
Epoch: 6; Model Loss: 5.148350715637207
Epoch: 7; Model Loss: 5.100292205810547
Epoch: 8; Model Loss: 5.053269863128662
Epoch: 9; Model Loss: 5.007540702819824
Epoch: 10; Model Loss: 4.962957859039307
Epoch: 11; Model Loss: 4.91948127746582
Epoch: 12; Model Loss: 4.876903533935547
Epoch: 13; Model Loss: 4.835084438323975
Epoch: 14; Model Loss: 4.794074535369873
Epoch: 15; Model Loss: 4.753962516784668
Epoch: 16; Model Loss: 4.714903831481934
Epoch: 17; Model Loss: 4.6767683029174805
Epoch: 18; Model Loss: 4.639404296875
Epoch: 19; Model Loss: 4.602801322937012
Epoch: 20; Model Loss: 4.567200183868408
Epoch: 21; Model Loss: 4.532393455505371
Epoch: 22; Model Loss: 4.498125076293945
Epoch: 23; Model Loss: 4.464578628540039
Epoch: 24; Model Loss: 4.4314